In [1]:
import pandas as pd
import numpy as np
from lightfm.data import Dataset
from lightfm.cross_validation import random_train_test_split
from lightfm.evaluation import precision_at_k
from lightfm import LightFM

from hyperopt import fmin, hp, tpe, Trials
from scipy.io import mmwrite

C:\Users\SSAFY\AppData\Local\Programs\Python\Python39\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


In [2]:
pd.set_option('display.max_columns', None)

In [3]:
size_cloth_column = ['newClothId', 'clothMale', 'clothFemale', 'clothRate', 'clothReviewCnt', 'userHeight', 'userWeight', 'userMale', 'userFemale']
size_user_column = ['userId','userMale', 'userFemale', 'userHeight', 'userWeight']
favor_ctop_column = ['newClothId', 'smallCategoryHalfshort', 'smallCategoryShirt', 'smallCategoryCollar', 'smallCategoryHoody', 'smallCategorySweatshirt', 'smallCategoryKnit',
        'smallCategoryLong', 'smallCategoryShort', 'smallCategoryOthers', 'colorWhite', 'colorGrey', 'colorBlack', 'colorRed', 'colorPink', 'colorOrange', 'colorIvory', 'colorYellow',
        'colorGreen', 'colorBlue', 'colorPurple', 'colorBrown', 'colorBeige', 'colorJean', 'colorPattern', 'colorOthers', 'clothPrice', 'clothRate', 
        'clothReviewCnt', 'fit', 'feeling', 'stretch', 'visibility', 'thickness', 'userMale', 'userFemale', 'month1', 'month2', 'month3', 'month4', 'month5', 'month6', 'month7', 'month8', 'month9', 'month10', 'month11', 'month12']
favor_couter_column = ['newClothId', 'smallCategoryHoodie', 'smallCategoryBlouson', 'smallCategoryRiders', 'smallCategoryMustang', 'smallCategoryCardigan', 'smallCategoryFleece',
        'smallCategoryCoat', 'smallCategoryPaddedcoat', 'smallCategoryVest', 'smallCategoryJacket', 'smallCategoryOthers', 'colorWhite', 'colorGrey', 'colorBlack', 'colorRed', 'colorPink', 'colorOrange', 'colorIvory', 'colorYellow',
        'colorGreen', 'colorBlue', 'colorPurple', 'colorBrown', 'colorBeige', 'colorJean', 'colorPattern', 'colorOthers', 'clothPrice', 'clothRate', 
        'clothReviewCnt', 'fit', 'feeling', 'stretch', 'visibility', 'thickness', 'month1', 'month2', 'month3', 'month4', 'month5', 'month6', 'month7', 'month8', 'month9', 'month10', 'month11', 'month12']
favor_cpants_column = ['newClothId', 'smallCategoryDenimpants', 'smallCategoryCottonpants', 'smallCategorySlacks', 'smallCategoryJoggerpants', 'smallCategoryShortpants', 'smallCategoryLeggings',
       'smallCategoryJumpsuit', 'smallCategoryOthers', 'colorWhite', 'colorGrey', 'colorBlack', 'colorRed', 'colorPink', 'colorOrange', 'colorIvory', 'colorYellow',
       'colorGreen', 'colorBlue', 'colorPurple', 'colorBrown', 'colorBeige', 'colorJean', 'colorPattern', 'colorOthers', 'clothPrice', 'clothRate', 
       'clothReviewCnt', 'fit', 'feeling', 'stretch', 'visibility', 'thickness', 'month1', 'month2', 'month3', 'month4', 'month5', 'month6', 'month7', 'month8', 'month9', 'month10', 'month11', 'month12']
favor_conepiece_column = ['newClothId', 'smallCategoryMinidress', 'smallCategoryMidi', 'smallCategoryMaxidress', 'colorWhite', 'colorGrey', 'colorBlack', 'colorRed', 'colorPink', 'colorOrange', 'colorIvory', 'colorYellow',
       'colorGreen', 'colorBlue', 'colorPurple', 'colorBrown', 'colorBeige', 'colorJean', 'colorPattern', 'colorOthers', 'clothPrice', 'clothRate', 
       'clothReviewCnt', 'fit', 'feeling', 'stretch', 'visibility', 'thickness', 'month1', 'month2', 'month3', 'month4', 'month5', 'month6', 'month7', 'month8', 'month9', 'month10', 'month11', 'month12']
favor_cskirt_column = ['newClothId', 'smallCategoryMiniskirt', 'smallCategoryMidi', 'smallCategoryLongskirt', 'colorWhite', 'colorGrey', 'colorBlack', 'colorRed', 'colorPink', 'colorOrange', 'colorIvory', 'colorYellow',
       'colorGreen', 'colorBlue', 'colorPurple', 'colorBrown', 'colorBeige', 'colorJean', 'colorPattern', 'colorOthers', 'clothPrice', 'clothRate', 
       'clothReviewCnt', 'fit', 'feeling', 'stretch', 'visibility', 'thickness', 'month1', 'month2', 'month3', 'month4', 'month5', 'month6', 'month7', 'month8', 'month9', 'month10', 'month11', 'month12']
favor_utop_column = ['userId', 'smallCategoryHalfshort', 'smallCategoryShirt',
       'smallCategoryCollar', 'smallCategoryHoody', 'smallCategorySweatshirt', 'smallCategoryKnit', 'smallCategoryLong', 'smallCategoryShort',
       'smallCategoryOthers']
favor_uouter_column = ['userId', 
       'smallCategoryHoodie', 'smallCategoryBlouson', 'smallCategoryRiders', 'smallCategoryMustang', 'smallCategoryCardigan', 'smallCategoryFleece',
        'smallCategoryCoat', 'smallCategoryPaddedcoat', 'smallCategoryVest', 'smallCategoryJacket', 'smallCategoryOthers']
favor_upants_column = ['userId', 'smallCategoryDenimpants', 'smallCategoryCottonpants', 'smallCategorySlacks', 'smallCategoryJoggerpants', 'smallCategoryShortpants', 'smallCategoryLeggings',
       'smallCategoryJumpsuit', 'smallCategoryOthers']
favor_uonepiece_column = ['userId', 'smallCategoryMinidress', 'smallCategoryMidi', 'smallCategoryMaxidress']
favor_uskirt_column = ['userId', 'smallCategoryMiniskirt', 'smallCategoryMidi', 'smallCategoryLongskirt']
color_utop_column = ['userId', 'colorWhite', 'colorGrey', 'colorBlack', 'colorRed', 'colorPink', 'colorOrange', 'colorIvory', 'colorYellow',
       'colorGreen', 'colorBlue', 'colorPurple', 'colorBrown', 'colorBeige', 'colorJean', 'colorPattern', 'colorOthers']
color_uouter_column = ['userId', 'colorWhite', 'colorGrey', 'colorBlack',
       'colorRed', 'colorPink', 'colorOrange', 'colorIvory', 'colorYellow', 'colorGreen', 'colorBlue', 'colorPurple', 'colorBrown', 'colorBeige',
       'colorJean', 'colorPattern', 'colorOthers']
color_upants_column = ['userId', 'colorWhite', 'colorGrey', 'colorBlack', 'colorRed', 'colorPink', 'colorOrange', 'colorIvory', 'colorYellow',
       'colorGreen', 'colorBlue', 'colorPurple', 'colorBrown', 'colorBeige', 'colorJean', 'colorPattern', 'colorOthers']
color_uonepiece_column = ['userId', 'colorWhite', 'colorGrey', 'colorBlack',
       'colorRed', 'colorPink', 'colorOrange', 'colorIvory', 'colorYellow', 'colorGreen', 'colorBlue', 'colorPurple', 'colorBrown', 'colorBeige',
       'colorJean', 'colorPattern', 'colorOthers']
color_uskirt_column = ['userId', 'colorWhite', 'colorGrey', 'colorBlack',
       'colorRed', 'colorPink', 'colorOrange', 'colorIvory', 'colorYellow', 'colorGreen', 'colorBlue', 'colorPurple', 'colorBrown', 'colorBeige',
       'colorJean', 'colorPattern', 'colorOthers']
style_utop_column = ['userId', 'size', 'bright', 'color', 'thickness']
style_uouter_column = ['userId', 'size', 'bright', 'color', 'thickness']
style_upants_column = ['userId', 'size', 'bright', 'color', 'thickness']
style_uonepiece_column = ['userId', 'size', 'bright', 'color', 'thickness']
style_uskirt_column = ['userId', 'size', 'bright', 'color', 'thickness']

In [453]:
transaction = pd.read_json('./transaction_top.json')
cloth_meta = pd.read_json('./final_top.json')
user_meta = pd.read_json('./user_top.json')
transaction = transaction[['userId', 'newClothId', 'shopCnt']]

In [454]:
transaction.shape

(281090, 3)

In [455]:
unique_id_cloth_meta = list(set(cloth_meta.newClothId))
transaction=transaction.loc[transaction.newClothId.isin(unique_id_cloth_meta),:]

In [456]:
a = transaction[['newClothId']]
a.insert(1, 'idx', 0, True)
a = a.drop_duplicates(['newClothId'])
a.idx = range(len(a))

In [457]:
transaction = pd.merge(transaction, a, how="left", on="newClothId")

In [458]:
user_meta = user_meta[size_user_column]

In [459]:
unique_id_user_meta = list(set(transaction.userId))
user_meta=user_meta.loc[user_meta.userId.isin(unique_id_user_meta),:]
user_meta = user_meta.reset_index(drop=True)
user_meta

,userId,userMale,userFemale,userHeight,userWeight
0,858064,1,0,183,90
1,858065,0,1,164,57
2,858067,0,1,172,59
3,858070,1,0,167,70
4,858071,1,0,177,68
...,...,...,...,...,...
131310,1416935,1,0,183,77
131311,1416936,1,0,179,78
131312,1416938,0,1,171,60
131313,1416946,1,0,175,62


In [460]:
new_user = pd.DataFrame(user_meta.loc[0]).transpose()
new_user_idx = new_user.index
new_user.loc[new_user_idx, ('userId')] = 8299
new_user.loc[new_user_idx, ('userMale')] = 0
new_user.loc[new_user_idx, ('userFemale')] = 1
new_user.loc[new_user_idx, ('userHeight')] = 180
new_user.loc[new_user_idx, ('userWeight')] = 65
new_user = new_user[['userId', 'userMale', 'userFemale', 'userHeight', 'userWeight']]
new_user

,userId,userMale,userFemale,userHeight,userWeight
0,8299,0,1,180,65


In [461]:
sub_user = user_meta[user_meta.userFemale==new_user.userFemale.iloc[0]]

In [462]:
sub_user.userHeight = sub_user.apply(lambda x: abs(x.userHeight-new_user.userHeight.iloc[0]), axis=1)
sub_user.userWeight = sub_user.apply(lambda x: abs(x.userWeight-new_user.userWeight.iloc[0]), axis=1)

C:\Users\SSAFY\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [463]:
sub_user.insert(3, 'sizeSum', 0, True)
sub_user.sizeSum = sub_user.userHeight + sub_user.userWeight
sub_user

,userId,userMale,userFemale,sizeSum,userHeight,userWeight
1,858065,0,1,24,16,8
2,858067,0,1,14,8,6
6,858074,0,1,42,24,18
8,858081,0,1,32,18,14
10,858087,0,1,26,17,9
...,...,...,...,...,...,...
131302,1416900,0,1,24,11,13
131303,1416901,0,1,26,12,14
131305,1416919,0,1,37,22,15
131308,1416929,0,1,22,12,10


In [464]:
real_user = []
for i in range(3):
    real_user.append(sub_user.sort_values('sizeSum').iloc[i].userId)
real_user = [int(i) for i in real_user]
print(real_user)

[1281758, 968344, 1110912]


In [465]:
sub_user

,userId,userMale,userFemale,sizeSum,userHeight,userWeight
1,858065,0,1,24,16,8
2,858067,0,1,14,8,6
6,858074,0,1,42,24,18
8,858081,0,1,32,18,14
10,858087,0,1,26,17,9
...,...,...,...,...,...,...
131302,1416900,0,1,24,11,13
131303,1416901,0,1,26,12,14
131305,1416919,0,1,37,22,15
131308,1416929,0,1,22,12,10


In [466]:
cloth_meta = cloth_meta[size_cloth_column]

In [467]:
cloth_meta = pd.merge(cloth_meta, a, how="left", on="newClothId")

In [468]:
unique_cloth_id = list(transaction['newClothId'].unique())
len(unique_cloth_id)

59819

In [469]:
a = cloth_meta.newClothId.isin(unique_cloth_id)
cloth_meta = cloth_meta[a]
cloth_meta.index = range(len(cloth_meta))

In [470]:
cloth_meta.idx = cloth_meta.idx.astype('int')

In [471]:
transaction_source = [(transaction['userId'][i], transaction['idx'][i]) for i in range(transaction.shape[0])]

In [472]:
print('userId의 유니크 개수 :', len(transaction.userId.unique()))
print('clothId의 유니크 개수 :', len(transaction.idx.unique()))
print('user테이블의 userId의 유니크 개수 :', len(user_meta.userId.unique()))
print('cloth테이블의 clothId의 유니크 개수 :', len(cloth_meta.idx.unique()))

userId의 유니크 개수 : 131315
clothId의 유니크 개수 : 59819
user테이블의 userId의 유니크 개수 : 131315
cloth테이블의 clothId의 유니크 개수 : 59819


In [339]:
user_features_source = [(user_meta['userId'][i],
                        [user_meta['userMale'][i], user_meta['userFemale'][i], user_meta['userHeight'][i],
                        user_meta['userWeight'][i]]) for i in range(user_meta.shape[0])]

In [340]:
cloth_features_source = [(cloth_meta['idx'][i],
                        [cloth_meta['clothMale'][i], cloth_meta['clothFemale'][i],
                         cloth_meta['userHeight'][i], cloth_meta['userWeight'][i] ]) for i in range(cloth_meta.shape[0])]

In [341]:
dataset = Dataset()
dataset.fit(users=transaction['userId'].unique(),
           items=transaction['idx'].unique(),
           user_features=user_meta[user_meta.columns[1:]].values.flatten(),
           item_features=cloth_meta[cloth_meta.columns[1:]].values.flatten())

In [342]:
interactions, weights = dataset.build_interactions(transaction_source)

In [343]:
user_features = dataset.build_user_features(user_features_source)

In [29]:
cloth_features = dataset.build_item_features(cloth_features_source)

In [30]:
train, test = random_train_test_split(interactions, test_percentage=0.2)
train, test = train.tocsr().tocoo(), test.tocsr().tocoo()
train_weights = train.multiply(weights).tocoo()
test_weights = test.multiply(weights).tocoo()

In [31]:
# no_components: 10
# learning_rate : 0.05
trials = Trials()
space = [hp.choice('no_components', range(10, 21, 5)),
         hp.uniform('learning_rate', 0.05, 0.1)]

In [85]:
# Define Objective Function
def objective(params):
    no_components, learning_rate = params
    model = LightFM(no_components=no_components,
                    learning_schedule='adagrad',
                    loss='warp',
                    learning_rate=learning_rate,
                    random_state=0)

    model.fit(interactions=train,
              item_features=cloth_features,
              user_features=user_features,
              sample_weight=train_weights,
              epochs=3,
              verbose=False)

    test_precision = precision_at_k(model, test, k=5, item_features=cloth_features, user_features=user_features).mean()
    print("no_comp: {}, lrn_rate: {:.5f}, precision: {:.5f}".format(
      no_components, learning_rate, test_precision))
    # test_auc = auc_score(model, test, item_features=item_features).mean()
    output = -test_precision

    if np.abs(output+1) < 0.01 or output < -1.0:
        output = 0.0

    return output

In [ ]:
best_params = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=10, trials=trials)

  0%|          | 0/10 [00:00<?, ?trial/s, best loss=?]

In [31]:
learning_rate = round(best_params['learning_rate'], 3)
no_components = best_params['no_components']
print(learning_rate, no_components)

0.05 2


In [31]:
learning_rate = 0.03
no_components = 3

In [79]:
model = LightFM(no_components=no_components,
                learning_schedule='adagrad',
                loss='warp',
                learning_rate=learning_rate,
                random_state=0)

model.fit(interactions=interactions,
          item_features=cloth_features,
          user_features=user_features,
          sample_weight=weights,
          epochs=10,
          verbose=True)

Epoch: 100%|██████████| 10/10 [00:09<00:00,  1.01it/s]


In [80]:
# 모델 저장하기
import pickle
with open('model_size_top.pickle', 'wb') as fw:
    pickle.dump(model, fw)

In [65]:
# 모델 불러오기
import pickle
with open('model_size_top.pickle', 'rb') as f:
    model_top = pickle.load(f)

In [82]:
def sample_recommendation(model, idx, user_ids):
    cnt = 1
    for user_id in user_ids:
        if cnt==1:
            scores = model.predict(int(user_meta[user_meta.userId==user_id].index[0]), np.arange(len(cloth_meta)))
        else:
            scores += model.predict(int(user_meta[user_meta.userId==user_id].index[0]), np.arange(len(cloth_meta)))
    
    top_items = idx[np.argsort(-scores)]
    
    sub_clothes = []

    for x in top_items[:10]:
        sub_clothes.append(x)
    
    rec_clothes = []
    
    for i in sub_clothes:
        rec_clothes.append(cloth_meta[cloth_meta.idx==i].newClothId.iloc[0])
    
    return rec_clothes

In [83]:
rec_size_top = sample_recommendation(model_top, cloth_meta.idx, real_user)
rec_size_top

[1440587133960,
 2139133140300,
 2155926132332,
 1572660131416,
 1804099134922,
 2139133140305,
 1572660131415,
 1282892131862,
 1938231133669,
 2139133140268]

In [60]:
real_user

[1281758, 968344, 1110912]

In [32]:
cloth_meta.insert(10, 'what', 1, True)
user_meta.insert(5, 'what', 1, True)

In [33]:
cloth_meta = cloth_meta[["newClothId", "idx", 'what']]
cloth_meta

,newClothId,idx,what
0,1440513131686,50031,1
1,1930909131765,7019,1
2,1930909131764,7840,1
3,1930909131766,7374,1
4,1931411131771,9496,1
...,...,...,...
59814,1281638267701,31274,1
59815,1622670267773,9458,1
59816,1556851267795,47549,1
59817,1651849267836,3995,1


In [476]:
cloth_meta.to_json('size_top_cloth_meta.json', orient='records', force_ascii=False)
user_meta.to_json('size_top_user_meta.json', orient='records', force_ascii=False)

In [473]:
# user_meta.insert(0, 'how', 1, True)
user_meta.insert(0, 'what', 1, True)

In [474]:
user_meta = user_meta.drop(['userMale', 'userFemale', 'userHeight', 'userWeight'], axis=1)

In [475]:
user_meta

,what,userId
0,1,858064
1,1,858065
2,1,858067
3,1,858070
4,1,858071
...,...,...
131310,1,1416935
131311,1,1416936
131312,1,1416938
131313,1,1416946


# Outer

In [4]:
transaction = pd.read_json('./transaction_outer.json')
cloth_meta = pd.read_json('./final_outer.json')
user_meta = pd.read_json('./user_outer.json')

In [23]:
transaction

,userId,newClothId,largecategory,shopCnt,idx
0,488503,117338595070,2,2,0
1,488504,168749091775,2,2,1
2,488505,177840491505,2,1,2
3,488505,208909488667,2,1,3
4,488505,2087957114719,2,1,4
...,...,...,...,...,...
707759,858060,208083683393,2,2,2941
707760,858060,208084081907,2,2,8397
707761,858061,175920483334,2,1,39318
707762,858061,1865226121614,2,1,52543


In [5]:
unique_id_cloth_meta = list(set(cloth_meta.newClothId))
transaction=transaction.loc[transaction.newClothId.isin(unique_id_cloth_meta),:]

In [6]:
a = transaction[['newClothId']]
a.insert(1, 'idx', 0, True)
a = a.drop_duplicates(['newClothId'])
a.idx = range(len(a))

In [7]:
transaction = pd.merge(transaction, a, how="left", on="newClothId")

In [8]:
user_meta = user_meta[size_user_column]

In [9]:
unique_id_user_meta = list(set(transaction.userId))
user_meta=user_meta.loc[user_meta.userId.isin(unique_id_user_meta),:]
user_meta = user_meta.reset_index(drop=True)
user_meta

,userId,userMale,userFemale,userHeight,userWeight
0,488503,1,0,164,50
1,488504,0,1,164,57
2,488505,1,0,175,55
3,488506,0,1,172,62
4,488507,1,0,177,68
...,...,...,...,...,...
369515,858058,1,0,179,78
369516,858059,0,1,171,60
369517,858060,1,0,169,57
369518,858061,1,0,175,60


In [10]:
new_user = pd.DataFrame(user_meta.loc[0]).transpose()
new_user_idx = new_user.index
new_user.loc[new_user_idx, ('userId')] = 8299
new_user.loc[new_user_idx, ('userMale')] = 0
new_user.loc[new_user_idx, ('userFemale')] = 1
new_user.loc[new_user_idx, ('userHeight')] = 180
new_user.loc[new_user_idx, ('userWeight')] = 65
new_user = new_user[['userId', 'userMale', 'userFemale', 'userHeight', 'userWeight']]
new_user

,userId,userMale,userFemale,userHeight,userWeight
0,8299,0,1,180,65


In [11]:
sub_user = user_meta[user_meta.userFemale==new_user.userFemale.iloc[0]]

In [12]:
sub_user.userHeight = sub_user.apply(lambda x: abs(x.userHeight-new_user.userHeight.iloc[0]), axis=1)
sub_user.userWeight = sub_user.apply(lambda x: abs(x.userWeight-new_user.userWeight.iloc[0]), axis=1)

C:\Users\SSAFY\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [13]:
sub_user.insert(3, 'sizeSum', 0, True)
sub_user.sizeSum = sub_user.apply(lambda x: x.userHeight + x.userWeight, axis=1)

In [14]:
real_user = []
for i in range(3):
    real_user.append(sub_user.sort_values('sizeSum').iloc[i].userId)
real_user = [int(i) for i in real_user]
print(real_user)

[794678, 654568, 850741]


In [15]:
cloth_meta = cloth_meta[size_cloth_column]

In [16]:
cloth_meta = pd.merge(cloth_meta, a, how="left", on="newClothId")

In [17]:
unique_cloth_id = list(transaction['newClothId'].unique())
len(unique_cloth_id)

52544

In [18]:
a = cloth_meta.newClothId.isin(unique_cloth_id)
cloth_meta = cloth_meta[a]
cloth_meta.index = range(len(cloth_meta))

In [19]:
cloth_meta.idx = cloth_meta.idx.astype('int')

In [20]:
transaction_source = [(transaction['userId'][i], transaction['idx'][i]) for i in range(transaction.shape[0])]

In [21]:
print('userId의 유니크 개수 :', len(transaction.userId.unique()))
print('clothId의 유니크 개수 :', len(transaction.idx.unique()))
print('user테이블의 userId의 유니크 개수 :', len(user_meta.userId.unique()))
print('cloth테이블의 clothId의 유니크 개수 :', len(cloth_meta.idx.unique()))

userId의 유니크 개수 : 369520
clothId의 유니크 개수 : 52544
user테이블의 userId의 유니크 개수 : 369520
cloth테이블의 clothId의 유니크 개수 : 52544


In [370]:
user_features_source = [(user_meta['userId'][i],
                        [user_meta['userMale'][i], user_meta['userFemale'][i], user_meta['userHeight'][i],
                        user_meta['userWeight'][i]]) for i in range(user_meta.shape[0])]

In [371]:
cloth_features_source = [(cloth_meta['idx'][i],
                        [cloth_meta['clothMale'][i], cloth_meta['clothFemale'][i],
                         cloth_meta['userHeight'][i], cloth_meta['userWeight'][i] ]) for i in range(cloth_meta.shape[0])]

In [372]:
dataset = Dataset()
dataset.fit(users=transaction['userId'].unique(),
           items=transaction['idx'].unique(),
           user_features=user_meta[user_meta.columns[1:]].values.flatten(),
           item_features=cloth_meta[cloth_meta.columns[1:]].values.flatten())

In [373]:
interactions, weights = dataset.build_interactions(transaction_source)

In [374]:
user_features = dataset.build_user_features(user_features_source)

In [375]:
cloth_features = dataset.build_item_features(cloth_features_source)

In [376]:
train, test = random_train_test_split(interactions, test_percentage=0.2)
train, test = train.tocsr().tocoo(), test.tocsr().tocoo()
train_weights = train.multiply(weights).tocoo()
test_weights = test.multiply(weights).tocoo()

In [377]:
# no_components: 10
# learning_rate : 0.05
trials = Trials()
space = [hp.choice('no_components', range(10, 21, 5)),
         hp.uniform('learning_rate', 0.05, 0.1)]

In [378]:
# Define Objective Function
def objective(params):
    no_components, learning_rate = params
    model = LightFM(no_components=no_components,
                    learning_schedule='adagrad',
                    loss='warp',
                    learning_rate=learning_rate,
                    random_state=0)

    model.fit(interactions=train,
              item_features=cloth_features,
              user_features=user_features,
              sample_weight=train_weights,
              epochs=3,
              verbose=False)

    test_precision = precision_at_k(model, test, k=5, item_features=cloth_features, user_features=user_features).mean()
    print("no_comp: {}, lrn_rate: {:.5f}, precision: {:.5f}".format(
      no_components, learning_rate, test_precision))
    # test_auc = auc_score(model, test, item_features=item_features).mean()
    output = -test_precision

    if np.abs(output+1) < 0.01 or output < -1.0:
        output = 0.0

    return output

In [97]:
best_params = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=10, trials=trials)

  0%|          | 0/10 [01:05<?, ?trial/s, best loss=?]


KeyboardInterrupt: 

In [31]:
learning_rate = round(best_params['learning_rate'], 3)
no_components = best_params['no_components']
print(learning_rate, no_components)

0.05 2


In [163]:
learning_rate = 0.03
no_components = 3

In [164]:
model = LightFM(no_components=no_components,
                learning_schedule='adagrad',
                loss='warp',
                learning_rate=learning_rate,
                random_state=0)

model.fit(interactions=interactions,
          item_features=cloth_features,
          user_features=user_features,
          sample_weight=weights,
          epochs=10,
          verbose=True)

Epoch: 100%|██████████| 10/10 [00:05<00:00,  1.94it/s]


In [165]:
# 모델 저장하기
import pickle
with open('model_size_outer.pickle', 'wb') as fw:
    pickle.dump(model, fw)

In [166]:
# 모델 불러오기
import pickle
with open('model_size_outer.pickle', 'rb') as f:
    model_outer = pickle.load(f)

In [167]:
def sample_recommendation(model, idx, user_ids):
    cnt = 1
    for user_id in user_ids:
        if cnt==1:
            scores = model.predict(int(user_meta[user_meta.userId==user_id].index[0]), np.arange(len(cloth_meta)))
        else:
            scores += model.predict(int(user_meta[user_meta.userId==user_id].index[0]), np.arange(len(cloth_meta)))
    
    top_items = idx[np.argsort(-scores)]
    
    sub_clothes = []

    for x in top_items[:10]:
        sub_clothes.append(x)
    
    rec_clothes = []
    
    for i in sub_clothes:
        rec_clothes.append(cloth_meta[cloth_meta.idx==i].newClothId.iloc[0])
    
    return rec_clothes

In [168]:
rec_size_top = sample_recommendation(model_outer, cloth_meta.idx, real_user)
rec_size_top

[1855501110996,
 208155079212,
 162907498408,
 157348798846,
 177400380239,
 161198282328,
 236907099750,
 1577133111047,
 231905785481,
 235944795060]

In [169]:
real_user

[708919, 558795, 538194]

In [170]:
cloth_meta.insert(10, 'what', 2, True)
user_meta.insert(5, 'what', 2, True)

In [171]:
cloth_meta = cloth_meta[["newClothId", "idx", 'what']]
cloth_meta

,newClothId,idx,what
0,234005278848,25240,2
1,208155079211,20700,2
2,208155079212,15426,2
3,208155079213,23065,2
4,208155079214,1832,2
...,...,...,...
25911,2080918130502,10939,2
25912,2080918130504,7261,2
25913,2322135130506,21919,2
25914,2322135130507,20550,2


In [383]:
cloth_meta.to_json('size_outer_cloth_meta.json', orient='records', force_ascii=False)
user_meta.to_json('size_outer_user_meta.json', orient='records', force_ascii=False)

In [379]:
# user_meta.insert(0, 'how', 2, True)
user_meta.insert(0, 'what', 2, True)

In [381]:
user_meta = user_meta.drop(['userMale', 'userFemale', 'userHeight', 'userWeight'], axis=1)

In [382]:
user_meta

,what,userId
0,2,488503
1,2,488504
2,2,488506
3,2,488507
4,2,488509
...,...,...
93245,2,858056
93246,2,858057
93247,2,858058
93248,2,858060


# Pants

In [24]:
transaction = pd.read_json('./transaction_pants.json')
cloth_meta = pd.read_json('./final_pants.json')
user_meta = pd.read_json('./user_pants.json')

In [25]:
transaction

,userId,newClothId,largecategory,shopCnt
0,14134,18803848686,3,1
1,14134,213219072452,3,1
2,14135,155184048154,3,1
3,14135,167945567571,3,2
4,14135,193152643836,3,1
...,...,...,...,...
1225285,488500,999449069845,3,1
1225286,488501,147727234807,3,1
1225287,488501,188543376408,3,1
1225288,488502,101577772513,3,2


In [385]:
unique_id_cloth_meta = list(set(cloth_meta.newClothId))
transaction=transaction.loc[transaction.newClothId.isin(unique_id_cloth_meta),:]

In [386]:
a = transaction[['newClothId']]
a.insert(1, 'idx', 0, True)
a = a.drop_duplicates(['newClothId'])
a.idx = range(len(a))

In [387]:
transaction = pd.merge(transaction, a, how="left", on="newClothId")

In [388]:
user_meta = user_meta[size_user_column]

In [389]:
unique_id_user_meta = list(set(transaction.userId))
user_meta=user_meta.loc[user_meta.userId.isin(unique_id_user_meta),:]
user_meta = user_meta.reset_index(drop=True)
user_meta

,userId,userMale,userFemale,userHeight,userWeight
0,14135,0,1,172,59
1,14141,1,0,182,77
2,14146,0,1,162,51
3,14148,1,0,180,68
4,14158,1,0,158,57
...,...,...,...,...,...
115960,488479,1,0,180,78
115961,488483,0,1,168,56
115962,488490,1,0,183,77
115963,488491,1,0,179,78


In [390]:
new_user = pd.DataFrame(user_meta.loc[0]).transpose()
new_user_idx = new_user.index
new_user.loc[new_user_idx, ('userId')] = 8299
new_user.loc[new_user_idx, ('userMale')] = 0
new_user.loc[new_user_idx, ('userFemale')] = 1
new_user.loc[new_user_idx, ('userHeight')] = 180
new_user.loc[new_user_idx, ('userWeight')] = 65
new_user = new_user[['userId', 'userMale', 'userFemale', 'userHeight', 'userWeight']]
new_user

,userId,userMale,userFemale,userHeight,userWeight
0,8299,0,1,180,65


In [391]:
sub_user = user_meta[user_meta.userFemale==new_user.userFemale.iloc[0]]

In [392]:
sub_user.userHeight = sub_user.apply(lambda x: abs(x.userHeight-new_user.userHeight.iloc[0]), axis=1)
sub_user.userWeight = sub_user.apply(lambda x: abs(x.userWeight-new_user.userWeight.iloc[0]), axis=1)

C:\Users\SSAFY\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [393]:
sub_user.insert(3, 'sizeSum', 0, True)
sub_user.sizeSum = sub_user.apply(lambda x: x.userHeight + x.userWeight, axis=1)

In [394]:
real_user = []
for i in range(3):
    real_user.append(sub_user.sort_values('sizeSum').iloc[i].userId)
real_user = [int(i) for i in real_user]
print(real_user)

[178769, 217810, 176013]


In [395]:
cloth_meta = cloth_meta[size_cloth_column]

In [396]:
cloth_meta = pd.merge(cloth_meta, a, how="left", on="newClothId")

In [397]:
unique_cloth_id = list(transaction['newClothId'].unique())
len(unique_cloth_id)

33530

In [398]:
a = cloth_meta.newClothId.isin(unique_cloth_id)
cloth_meta = cloth_meta[a]
cloth_meta.index = range(len(cloth_meta))

In [399]:
cloth_meta.idx = cloth_meta.idx.astype('int')

In [400]:
transaction_source = [(transaction['userId'][i], transaction['idx'][i]) for i in range(transaction.shape[0])]

In [401]:
print('userId의 유니크 개수 :', len(transaction.userId.unique()))
print('clothId의 유니크 개수 :', len(transaction.idx.unique()))
print('user테이블의 userId의 유니크 개수 :', len(user_meta.userId.unique()))
print('cloth테이블의 clothId의 유니크 개수 :', len(cloth_meta.idx.unique()))

userId의 유니크 개수 : 115965
clothId의 유니크 개수 : 33530
user테이블의 userId의 유니크 개수 : 115965
cloth테이블의 clothId의 유니크 개수 : 33530


In [402]:
user_features_source = [(user_meta['userId'][i],
                        [user_meta['userMale'][i], user_meta['userFemale'][i], user_meta['userHeight'][i],
                        user_meta['userWeight'][i]]) for i in range(user_meta.shape[0])]

In [403]:
cloth_features_source = [(cloth_meta['idx'][i],
                        [cloth_meta['clothMale'][i], cloth_meta['clothFemale'][i],
                         cloth_meta['userHeight'][i], cloth_meta['userWeight'][i] ]) for i in range(cloth_meta.shape[0])]

In [404]:
dataset = Dataset()
dataset.fit(users=transaction['userId'].unique(),
           items=transaction['idx'].unique(),
           user_features=user_meta[user_meta.columns[1:]].values.flatten(),
           item_features=cloth_meta[cloth_meta.columns[1:]].values.flatten())

In [196]:
interactions, weights = dataset.build_interactions(transaction_source)

In [197]:
user_features = dataset.build_user_features(user_features_source)

In [198]:
cloth_features = dataset.build_item_features(cloth_features_source)

In [199]:
train, test = random_train_test_split(interactions, test_percentage=0.2)
train, test = train.tocsr().tocoo(), test.tocsr().tocoo()
train_weights = train.multiply(weights).tocoo()
test_weights = test.multiply(weights).tocoo()

In [30]:
# no_components: 10
# learning_rate : 0.05
trials = Trials()
space = [hp.choice('no_components', range(10, 21, 5)),
         hp.uniform('learning_rate', 0.05, 0.1)]

In [85]:
# Define Objective Function
def objective(params):
    no_components, learning_rate = params
    model = LightFM(no_components=no_components,
                    learning_schedule='adagrad',
                    loss='warp',
                    learning_rate=learning_rate,
                    random_state=0)

    model.fit(interactions=train,
              item_features=cloth_features,
              user_features=user_features,
              sample_weight=train_weights,
              epochs=3,
              verbose=False)

    test_precision = precision_at_k(model, test, k=5, item_features=cloth_features, user_features=user_features).mean()
    print("no_comp: {}, lrn_rate: {:.5f}, precision: {:.5f}".format(
      no_components, learning_rate, test_precision))
    # test_auc = auc_score(model, test, item_features=item_features).mean()
    output = -test_precision

    if np.abs(output+1) < 0.01 or output < -1.0:
        output = 0.0

    return output

In [ ]:
best_params = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=10, trials=trials)

  0%|          | 0/10 [00:00<?, ?trial/s, best loss=?]

In [31]:
learning_rate = round(best_params['learning_rate'], 3)
no_components = best_params['no_components']
print(learning_rate, no_components)

0.05 2


In [200]:
learning_rate = 0.03
no_components = 3

In [201]:
model = LightFM(no_components=no_components,
                learning_schedule='adagrad',
                loss='warp',
                learning_rate=learning_rate,
                random_state=0)

model.fit(interactions=interactions,
          item_features=cloth_features,
          user_features=user_features,
          sample_weight=weights,
          epochs=10,
          verbose=True)

Epoch: 100%|██████████| 10/10 [00:08<00:00,  1.17it/s]


In [202]:
# 모델 저장하기
import pickle
with open('model_size_pants.pickle', 'wb') as fw:
    pickle.dump(model, fw)

In [203]:
# 모델 불러오기
import pickle
with open('model_size_pants.pickle', 'rb') as f:
    model_pants = pickle.load(f)

In [204]:
def sample_recommendation(model, idx, user_ids):
    cnt = 1
    for user_id in user_ids:
        if cnt==1:
            scores = model.predict(int(user_meta[user_meta.userId==user_id].index[0]), np.arange(len(cloth_meta)))
        else:
            scores += model.predict(int(user_meta[user_meta.userId==user_id].index[0]), np.arange(len(cloth_meta)))
    
    top_items = idx[np.argsort(-scores)]
    
    sub_clothes = []

    for x in top_items[:10]:
        sub_clothes.append(x)
    
    rec_clothes = []
    
    for i in sub_clothes:
        rec_clothes.append(cloth_meta[cloth_meta.idx==i].newClothId.iloc[0])
    
    return rec_clothes

In [205]:
rec_size_top = sample_recommendation(model_pants, cloth_meta.idx, real_user)
rec_size_top

[185616414367,
 20926756690,
 143216627295,
 166980648348,
 203291016298,
 222230112640,
 21061069363,
 20819699449,
 135793715512,
 120964158399]

In [60]:
real_user

[1281758, 968344, 1110912]

In [206]:
cloth_meta.insert(10, 'what', 3, True)
user_meta.insert(5, 'what', 3, True)

In [207]:
cloth_meta = cloth_meta[["newClothId", "idx", 'what']]
cloth_meta

,newClothId,idx,what
0,16225354884,11720,3
1,14988915210,15601,3
2,19409615513,29005,3
3,20193975542,22897,3
4,18466875743,17729,3
...,...,...,...
33525,180323169094,6877,3
33526,182037869103,19148,3
33527,194119769137,18084,3
33528,179115969145,16294,3


In [408]:
cloth_meta.to_json('size_pants_cloth_meta.json', orient='records', force_ascii=False)
user_meta.to_json('size_pants_user_meta.json', orient='records', force_ascii=False)

In [405]:
# user_meta.insert(0, 'how', 3, True)
user_meta.insert(0, 'what', 3, True)

In [406]:
user_meta = user_meta.drop(['userMale', 'userFemale', 'userHeight', 'userWeight'], axis=1)

In [407]:
user_meta

,what,userId
0,3,14135
1,3,14141
2,3,14146
3,3,14148
4,3,14158
...,...,...
115960,3,488479
115961,3,488483
115962,3,488490
115963,3,488491


# Onepiece

In [409]:
transaction = pd.read_json('./transaction_onepiece.json')
cloth_meta = pd.read_json('./final_onepiece.json')
user_meta = pd.read_json('./user_onepiece.json')

In [410]:
unique_id_cloth_meta = list(set(cloth_meta.newClothId))
transaction=transaction.loc[transaction.newClothId.isin(unique_id_cloth_meta),:]

In [411]:
a = transaction[['newClothId']]
a.insert(1, 'idx', 0, True)
a = a.drop_duplicates(['newClothId'])
a.idx = range(len(a))

In [412]:
transaction = pd.merge(transaction, a, how="left", on="newClothId")

In [413]:
user_meta = user_meta[size_user_column]

In [414]:
unique_id_user_meta = list(set(transaction.userId))
user_meta=user_meta.loc[user_meta.userId.isin(unique_id_user_meta),:]
user_meta = user_meta.reset_index(drop=True)
user_meta

,userId,userMale,userFemale,userHeight,userWeight
0,8297,0,1,160,49
1,8298,0,1,157,46
2,8299,0,1,161,50
3,8300,0,1,164,47
4,8301,0,1,164,50
...,...,...,...,...,...
5832,14129,0,1,168,56
5833,14130,0,1,166,51
5834,14131,0,1,160,50
5835,14132,0,1,166,48


In [415]:
new_user = pd.DataFrame(user_meta.loc[0]).transpose()
new_user_idx = new_user.index
new_user.loc[new_user_idx, ('userId')] = 8299
new_user.loc[new_user_idx, ('userMale')] = 0
new_user.loc[new_user_idx, ('userFemale')] = 1
new_user.loc[new_user_idx, ('userHeight')] = 180
new_user.loc[new_user_idx, ('userWeight')] = 65
new_user = new_user[['userId', 'userMale', 'userFemale', 'userHeight', 'userWeight']]
new_user

,userId,userMale,userFemale,userHeight,userWeight
0,8299,0,1,180,65


In [416]:
sub_user = user_meta[user_meta.userFemale==new_user.userFemale.iloc[0]]

In [417]:
sub_user.userHeight = sub_user.apply(lambda x: abs(x.userHeight-new_user.userHeight.iloc[0]), axis=1)
sub_user.userWeight = sub_user.apply(lambda x: abs(x.userWeight-new_user.userWeight.iloc[0]), axis=1)

C:\Users\SSAFY\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [418]:
sub_user.insert(3, 'sizeSum', 0, True)
sub_user.sizeSum = sub_user.apply(lambda x: x.userHeight + x.userWeight, axis=1)

In [419]:
real_user = []
for i in range(3):
    real_user.append(sub_user.sort_values('sizeSum').iloc[i].userId)
real_user = [int(i) for i in real_user]
print(real_user)

[10546, 8442, 13398]


In [420]:
cloth_meta = cloth_meta[size_cloth_column]

In [421]:
cloth_meta = pd.merge(cloth_meta, a, how="left", on="newClothId")

In [422]:
unique_cloth_id = list(transaction['newClothId'].unique())
len(unique_cloth_id)

2047

In [423]:
a = cloth_meta.newClothId.isin(unique_cloth_id)
cloth_meta = cloth_meta[a]
cloth_meta.index = range(len(cloth_meta))

In [424]:
cloth_meta.idx = cloth_meta.idx.astype('int')

In [425]:
transaction_source = [(transaction['userId'][i], transaction['idx'][i]) for i in range(transaction.shape[0])]

In [426]:
print('userId의 유니크 개수 :', len(transaction.userId.unique()))
print('clothId의 유니크 개수 :', len(transaction.idx.unique()))
print('user테이블의 userId의 유니크 개수 :', len(user_meta.userId.unique()))
print('cloth테이블의 clothId의 유니크 개수 :', len(cloth_meta.idx.unique()))

userId의 유니크 개수 : 5837
clothId의 유니크 개수 : 2047
user테이블의 userId의 유니크 개수 : 5837
cloth테이블의 clothId의 유니크 개수 : 2047


In [247]:
user_features_source = [(user_meta['userId'][i],
                        [user_meta['userMale'][i], user_meta['userFemale'][i], user_meta['userHeight'][i],
                        user_meta['userWeight'][i]]) for i in range(user_meta.shape[0])]

In [248]:
cloth_features_source = [(cloth_meta['idx'][i],
                        [cloth_meta['clothMale'][i], cloth_meta['clothFemale'][i],
                         cloth_meta['userHeight'][i], cloth_meta['userWeight'][i] ]) for i in range(cloth_meta.shape[0])]

In [249]:
dataset = Dataset()
dataset.fit(users=transaction['userId'].unique(),
           items=transaction['idx'].unique(),
           user_features=user_meta[user_meta.columns[1:]].values.flatten(),
           item_features=cloth_meta[cloth_meta.columns[1:]].values.flatten())

In [250]:
interactions, weights = dataset.build_interactions(transaction_source)

In [251]:
user_features = dataset.build_user_features(user_features_source)

In [252]:
cloth_features = dataset.build_item_features(cloth_features_source)

In [253]:
train, test = random_train_test_split(interactions, test_percentage=0.2)
train, test = train.tocsr().tocoo(), test.tocsr().tocoo()
train_weights = train.multiply(weights).tocoo()
test_weights = test.multiply(weights).tocoo()

In [261]:
# no_components: 10
# learning_rate : 0.05
trials = Trials()
space = [hp.choice('no_components', range(10, 50, 5)),
         hp.uniform('learning_rate', 0.05, 0.1)]

In [262]:
# Define Objective Function
def objective(params):
    no_components, learning_rate = params
    model = LightFM(no_components=no_components,
                    learning_schedule='adagrad',
                    loss='warp',
                    learning_rate=learning_rate,
                    random_state=0)

    model.fit(interactions=train,
              item_features=cloth_features,
              user_features=user_features,
              sample_weight=train_weights,
              epochs=3,
              verbose=False)

    test_precision = precision_at_k(model, test, k=5, item_features=cloth_features, user_features=user_features).mean()
    print("no_comp: {}, lrn_rate: {:.5f}, precision: {:.5f}".format(
      no_components, learning_rate, test_precision))
    # test_auc = auc_score(model, test, item_features=item_features).mean()
    output = -test_precision

    if np.abs(output+1) < 0.01 or output < -1.0:
        output = 0.0

    return output

In [263]:
best_params = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=10, trials=trials)

no_comp: 20, lrn_rate: 0.09204, precision: 0.01574    
no_comp: 15, lrn_rate: 0.05984, precision: 0.01304                                
no_comp: 15, lrn_rate: 0.08310, precision: 0.01274                                
no_comp: 25, lrn_rate: 0.06777, precision: 0.01304                                
no_comp: 20, lrn_rate: 0.06705, precision: 0.01319                                
no_comp: 35, lrn_rate: 0.07215, precision: 0.01484                                
no_comp: 15, lrn_rate: 0.07984, precision: 0.01274                                
no_comp: 30, lrn_rate: 0.05666, precision: 0.01364                                
no_comp: 10, lrn_rate: 0.06324, precision: 0.01394                                
no_comp: 25, lrn_rate: 0.08041, precision: 0.01304                                
100%|██████████| 10/10 [00:05<00:00,  1.69trial/s, best loss: -0.01574212871491909]


In [264]:
learning_rate = round(best_params['learning_rate'], 3)
no_components = best_params['no_components']
print(learning_rate, no_components)

0.092 2


In [265]:
model = LightFM(no_components=no_components,
                learning_schedule='adagrad',
                loss='warp',
                learning_rate=learning_rate,
                random_state=0)

model.fit(interactions=interactions,
          item_features=cloth_features,
          user_features=user_features,
          sample_weight=weights,
          epochs=10,
          verbose=True)

Epoch: 100%|██████████| 10/10 [00:00<00:00, 56.18it/s]


In [266]:
# 모델 저장하기
import pickle
with open('model_size_onepiece.pickle', 'wb') as fw:
    pickle.dump(model, fw)

In [267]:
# 모델 불러오기
import pickle
with open('model_size_onepiece.pickle', 'rb') as f:
    model_onepiece = pickle.load(f)

In [268]:
def sample_recommendation(model, idx, user_ids):
    cnt = 1
    for user_id in user_ids:
        if cnt==1:
            scores = model.predict(int(user_meta[user_meta.userId==user_id].index[0]), np.arange(len(cloth_meta)))
        else:
            scores += model.predict(int(user_meta[user_meta.userId==user_id].index[0]), np.arange(len(cloth_meta)))
    
    top_items = idx[np.argsort(-scores)]
    
    sub_clothes = []

    for x in top_items[:10]:
        sub_clothes.append(x)
    
    rec_clothes = []
    
    for i in sub_clothes:
        rec_clothes.append(cloth_meta[cloth_meta.idx==i].newClothId.iloc[0])
    
    return rec_clothes

In [269]:
rec_size_top = sample_recommendation(model_onepiece, cloth_meta.idx, real_user)
rec_size_top

[19345122958,
 21258303010,
 17957783204,
 19095243432,
 19186162758,
 15036583193,
 19952692865,
 17971332979,
 20163412966,
 23792962867]

In [270]:
real_user

[10546, 8442, 13398]

In [288]:
cloth_meta.insert(2, 'what', 4, True)
user_meta.insert(5, 'what', 4, True)

In [276]:
cloth_meta = cloth_meta[["newClothId", "idx", 'what']]
cloth_meta

,newClothId,idx,what,what
0,14094532674,279,4,1
1,14163822698,1239,4,1
2,18962342742,78,4,1
3,14829532807,1135,4,1
4,13617652829,1543,4,1
...,...,...,...,...
2042,20653784693,298,4,1
2043,13105704695,1719,4,1
2044,14406794698,837,4,1
2045,19412954699,1421,4,1


In [430]:
cloth_meta.to_json('size_onepiece_cloth_meta.json', orient='records', force_ascii=False)
user_meta.to_json('size_onepiece_user_meta.json', orient='records', force_ascii=False)

In [427]:
# user_meta.insert(0, 'how', 4, True)
user_meta.insert(0, 'what', 4, True)

In [428]:
user_meta = user_meta.drop(['userMale', 'userFemale', 'userHeight', 'userWeight'], axis=1)

In [429]:
user_meta

,what,userId
0,4,8297
1,4,8298
2,4,8299
3,4,8300
4,4,8301
...,...,...
5832,4,14129
5833,4,14130
5834,4,14131
5835,4,14132


# Skirt

In [431]:
transaction = pd.read_json('./transaction_skirt.json')
cloth_meta = pd.read_json('./final_skirt.json')
user_meta = pd.read_json('./user_skirt.json')

In [432]:
unique_id_cloth_meta = list(set(cloth_meta.newClothId))
transaction=transaction.loc[transaction.newClothId.isin(unique_id_cloth_meta),:]

In [433]:
a = transaction[['newClothId']]
a.insert(1, 'idx', 0, True)
a = a.drop_duplicates(['newClothId'])
a.idx = range(len(a))

In [434]:
transaction = pd.merge(transaction, a, how="left", on="newClothId")

In [435]:
user_meta = user_meta[size_user_column]

In [436]:
unique_id_user_meta = list(set(transaction.userId))
user_meta=user_meta.loc[user_meta.userId.isin(unique_id_user_meta),:]
user_meta = user_meta.reset_index(drop=True)
user_meta

,userId,userMale,userFemale,userHeight,userWeight
0,0,0,1,160,55
1,1,0,1,160,49
2,2,0,1,155,50
3,3,0,1,161,44
4,4,0,1,164,47
...,...,...,...,...,...
8292,8292,0,1,157,55
8293,8293,0,1,152,45
8294,8294,0,1,169,48
8295,8295,0,1,160,48


In [437]:
new_user = pd.DataFrame(user_meta.loc[0]).transpose()
new_user_idx = new_user.index
new_user.loc[new_user_idx, ('userId')] = 8299
new_user.loc[new_user_idx, ('userMale')] = 0
new_user.loc[new_user_idx, ('userFemale')] = 1
new_user.loc[new_user_idx, ('userHeight')] = 180
new_user.loc[new_user_idx, ('userWeight')] = 65
new_user = new_user[['userId', 'userMale', 'userFemale', 'userHeight', 'userWeight']]
new_user

,userId,userMale,userFemale,userHeight,userWeight
0,8299,0,1,180,65


In [438]:
sub_user = user_meta[user_meta.userFemale==new_user.userFemale.iloc[0]]

In [439]:
sub_user.userHeight = sub_user.apply(lambda x: abs(x.userHeight-new_user.userHeight.iloc[0]), axis=1)
sub_user.userWeight = sub_user.apply(lambda x: abs(x.userWeight-new_user.userWeight.iloc[0]), axis=1)

C:\Users\SSAFY\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [440]:
sub_user.insert(3, 'sizeSum', 0, True)
sub_user.sizeSum = sub_user.apply(lambda x: x.userHeight + x.userWeight, axis=1)

In [441]:
real_user = []
for i in range(3):
    real_user.append(sub_user.sort_values('sizeSum').iloc[i].userId)
real_user = [int(i) for i in real_user]
print(real_user)

[2574, 2057, 5855]


In [442]:
cloth_meta = cloth_meta[size_cloth_column]

In [443]:
cloth_meta = pd.merge(cloth_meta, a, how="left", on="newClothId")

In [444]:
unique_cloth_id = list(transaction['newClothId'].unique())
len(unique_cloth_id)

2654

In [445]:
a = cloth_meta.newClothId.isin(unique_cloth_id)
cloth_meta = cloth_meta[a]
cloth_meta.index = range(len(cloth_meta))

In [446]:
cloth_meta.idx = cloth_meta.idx.astype('int')

In [447]:
transaction_source = [(transaction['userId'][i], transaction['idx'][i]) for i in range(transaction.shape[0])]

In [448]:
print('userId의 유니크 개수 :', len(transaction.userId.unique()))
print('clothId의 유니크 개수 :', len(transaction.idx.unique()))
print('user테이블의 userId의 유니크 개수 :', len(user_meta.userId.unique()))
print('cloth테이블의 clothId의 유니크 개수 :', len(cloth_meta.idx.unique()))

userId의 유니크 개수 : 8297
clothId의 유니크 개수 : 2654
user테이블의 userId의 유니크 개수 : 8297
cloth테이블의 clothId의 유니크 개수 : 2654


In [310]:
user_features_source = [(user_meta['userId'][i],
                        [user_meta['userMale'][i], user_meta['userFemale'][i], user_meta['userHeight'][i],
                        user_meta['userWeight'][i]]) for i in range(user_meta.shape[0])]

In [311]:
cloth_features_source = [(cloth_meta['idx'][i],
                        [cloth_meta['clothMale'][i], cloth_meta['clothFemale'][i],
                         cloth_meta['userHeight'][i], cloth_meta['userWeight'][i] ]) for i in range(cloth_meta.shape[0])]

In [312]:
dataset = Dataset()
dataset.fit(users=transaction['userId'].unique(),
           items=transaction['idx'].unique(),
           user_features=user_meta[user_meta.columns[1:]].values.flatten(),
           item_features=cloth_meta[cloth_meta.columns[1:]].values.flatten())

In [313]:
interactions, weights = dataset.build_interactions(transaction_source)

In [314]:
user_features = dataset.build_user_features(user_features_source)

In [315]:
cloth_features = dataset.build_item_features(cloth_features_source)

In [316]:
train, test = random_train_test_split(interactions, test_percentage=0.2)
train, test = train.tocsr().tocoo(), test.tocsr().tocoo()
train_weights = train.multiply(weights).tocoo()
test_weights = test.multiply(weights).tocoo()

In [317]:
# no_components: 10
# learning_rate : 0.05
trials = Trials()
space = [hp.choice('no_components', range(10, 21, 5)),
         hp.uniform('learning_rate', 0.05, 0.1)]

In [318]:
# Define Objective Function
def objective(params):
    no_components, learning_rate = params
    model = LightFM(no_components=no_components,
                    learning_schedule='adagrad',
                    loss='warp',
                    learning_rate=learning_rate,
                    random_state=0)

    model.fit(interactions=train,
              item_features=cloth_features,
              user_features=user_features,
              sample_weight=train_weights,
              epochs=3,
              verbose=False)

    test_precision = precision_at_k(model, test, k=5, item_features=cloth_features, user_features=user_features).mean()
    print("no_comp: {}, lrn_rate: {:.5f}, precision: {:.5f}".format(
      no_components, learning_rate, test_precision))
    # test_auc = auc_score(model, test, item_features=item_features).mean()
    output = -test_precision

    if np.abs(output+1) < 0.01 or output < -1.0:
        output = 0.0

    return output

In [319]:
best_params = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=10, trials=trials)

no_comp: 10, lrn_rate: 0.08114, precision: 0.00837    
no_comp: 10, lrn_rate: 0.06698, precision: 0.00877                                
no_comp: 10, lrn_rate: 0.09739, precision: 0.01069                                 
no_comp: 20, lrn_rate: 0.07452, precision: 0.00817                                 
no_comp: 10, lrn_rate: 0.05717, precision: 0.00797                                 
no_comp: 15, lrn_rate: 0.05459, precision: 0.01079                                 
no_comp: 10, lrn_rate: 0.09188, precision: 0.01089                                 
no_comp: 15, lrn_rate: 0.05946, precision: 0.01200                                 
no_comp: 20, lrn_rate: 0.05167, precision: 0.01019                                 
no_comp: 10, lrn_rate: 0.09140, precision: 0.01130                                 
100%|██████████| 10/10 [00:07<00:00,  1.34trial/s, best loss: -0.012002017349004745]


In [320]:
learning_rate = round(best_params['learning_rate'], 3)
no_components = best_params['no_components']
print(learning_rate, no_components)

0.059 1


In [321]:
model = LightFM(no_components=no_components,
                learning_schedule='adagrad',
                loss='warp',
                learning_rate=learning_rate,
                random_state=0)

model.fit(interactions=interactions,
          item_features=cloth_features,
          user_features=user_features,
          sample_weight=weights,
          epochs=10,
          verbose=True)

Epoch: 100%|██████████| 10/10 [00:00<00:00, 52.35it/s]


In [322]:
# 모델 저장하기
import pickle
with open('model_size_skirt.pickle', 'wb') as fw:
    pickle.dump(model, fw)

In [323]:
# 모델 불러오기
import pickle
with open('model_size_skirt.pickle', 'rb') as f:
    model_skirt = pickle.load(f)

In [324]:
def sample_recommendation(model, idx, user_ids):
    cnt = 1
    for user_id in user_ids:
        if cnt==1:
            scores = model.predict(int(user_meta[user_meta.userId==user_id].index[0]), np.arange(len(cloth_meta)))
        else:
            scores += model.predict(int(user_meta[user_meta.userId==user_id].index[0]), np.arange(len(cloth_meta)))
    
    top_items = idx[np.argsort(-scores)]
    
    sub_clothes = []

    for x in top_items[:10]:
        sub_clothes.append(x)
    
    rec_clothes = []
    
    for i in sub_clothes:
        rec_clothes.append(cloth_meta[cloth_meta.idx==i].newClothId.iloc[0])
    
    return rec_clothes

In [325]:
rec_size_top = sample_recommendation(model_skirt, cloth_meta.idx, real_user)
rec_size_top

[1907111403,
 1926066515,
 1775973474,
 1969751297,
 180781478,
 2017517424,
 150926840,
 1618698368,
 2205663410,
 1834795393]

In [60]:
real_user

[1281758, 968344, 1110912]

In [326]:
cloth_meta.insert(10, 'what', 5, True)
user_meta.insert(5, 'what', 5, True)

In [327]:
cloth_meta = cloth_meta[["newClothId", "idx", 'what']]
cloth_meta

,newClothId,idx,what
0,161812522,1682,5
1,161812521,896,5
2,185310349,1662,5
3,1884867110,2304,5
4,1629303146,965,5
...,...,...,...
2649,21172562650,22,5
2650,21172562649,2,5
2651,21172562648,531,5
2652,23400702652,1441,5


In [452]:
cloth_meta.to_json('size_skirt_cloth_meta.json', orient='records', force_ascii=False)
user_meta.to_json('size_skirt_user_meta.json', orient='records', force_ascii=False)

In [449]:
# user_meta.insert(0, 'how', 5, True)
user_meta.insert(0, 'what', 5, True)

In [450]:
user_meta = user_meta.drop(['userMale', 'userFemale', 'userHeight', 'userWeight'], axis=1)

In [451]:
user_meta

,what,userId
0,5,0
1,5,1
2,5,2
3,5,3
4,5,4
...,...,...
8292,5,8292
8293,5,8293
8294,5,8294
8295,5,8295
